In [1]:
import numpy as np
# import umap
# import s3fs
import pandas as pd
import csv
import pyarrow
import boto3
import datetime
import matplotlib.pyplot as plt


In [50]:
from keras.datasets import imdb
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import keras

In [15]:
raw_scores_2000 = pd.read_parquet('boxscores_raw_2000-01.parquet')
raw_scores_2001 = pd.read_parquet('boxscores_raw_2001-02.parquet')
raw_scores_2002 = pd.read_parquet('boxscores_raw_2002-03.parquet')

In [4]:
# raw_scores_2000.iloc[:,45:].head()

In [52]:
four_factors = ['home_game', 'EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT',
                'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT', 'OPP_OREB_PCT']
four_factors = ['home_game', 'EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT',
                'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT', 'OPP_OREB_PCT',
               'AST_PCT', 'DREB_PCT', 'REB_PCT', 'USG_PCT', 'PIE',
               'PCT_FGA_3PT', 'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 'PCT_PTS_FB', 
                'PCT_PTS_FT', 'PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT', 'PCT_AST_2PM',
               'PCT_AST_3PM', 'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM']

info_cols = ['GAME_ID', 'TEAM_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST', 'PTS', 'ngame']

def generate_sequences(input_df, window_length=5):
    df = input_df.copy()
    
    df['ngame'] = df[['GAME_ID']].apply(lambda x: int(x.GAME_ID[-4:]) , axis=1)
    df['home_game'] = (df.TEAM_ID == df.HOME_TEAM_ID).astype(int)
    df = df[four_factors + info_cols]
    df.loc[:, 'GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'], format='%Y-%m-%d')

    team_a_sequences = np.empty((0, window_length, len(four_factors)))
    team_b_sequences = np.empty((0, window_length, len(four_factors)))
    targets = np.empty(0)
    home_inputs = np.empty(0)
    
    for game in df.ngame:
        home_team, away_team = df.loc[df.ngame == game, ['HOME_TEAM_ID', 'VISITOR_TEAM_ID']].values[0]
        
        #Keep current game to get target, then remove
        home_df = df.loc[(df.TEAM_ID == home_team) & (df.ngame <= game), :]
        away_df = df.loc[(df.TEAM_ID == away_team) & (df.ngame <= game), :]
        
        temp = df.loc[(df.TEAM_ID.isin([away_team, home_team])) & (df.ngame == game), :]
        
        home_win = int(home_df.loc[home_df.ngame == game, 'PTS'].values[0] > 
                       away_df.loc[away_df.ngame == game, 'PTS'].values[0])

        home_df = home_df.loc[df.ngame < game, :]
        away_df = away_df.loc[df.ngame < game, :]

        if home_df.shape[0] > window_length and away_df.shape[0] > window_length:
            
#             home_sequence = np.zeros((window_length, four_factors.shape[0]))
            home_sequence = home_df.iloc[:window_length, :][four_factors].values
            away_sequence = away_df.iloc[:window_length, :][four_factors].values

            team_a_sequences = np.append(team_a_sequences, [home_sequence], axis=0)
            team_b_sequences = np.append(team_b_sequences, [away_sequence], axis=0)
            home_inputs = np.append(home_inputs, [1])
            
            team_a_sequences = np.append(team_a_sequences, [away_sequence], axis=0)
            team_b_sequences = np.append(team_b_sequences, [home_sequence], axis=0)
            home_inputs = np.append(home_inputs, [0])
            
            targets = np.append(targets, [home_win]*2)
            
    return team_a_sequences, team_b_sequences, home_inputs, targets

Xa_2000, Xb_2000, home_inputs_2000, y_2000 = generate_sequences(raw_scores_2000, 30)
Xa_2001, Xb_2001, home_inputs_2001, y_2001 = generate_sequences(raw_scores_2001, 30)

# X_2002, y_2002, home_inputs_2002 = generate_sequences(raw_scores_2002, 30)
# generate_sequences(raw_scores_2001, 15)


In [30]:
X_comb = np.concatenate((X_2000, X_2001), axis=0)
y_comb = np.concatenate((y_2000, y_2001), axis=0)
home_inputs_comb = np.concatenate((home_inputs_2000, home_inputs_2001), axis=0)


In [57]:
def train_sequential(X_train_a, X_train_b, home_input_train, y_train,
                     X_test_a, X_test_b, home_input_test, y_test):
    np.random.seed(42)
        
    team_a = Input(shape=(X_train_a.shape[1], X_train_a.shape[2]))
    team_b = Input(shape=(X_train_b.shape[1], X_train_b.shape[2]))
    
    # model = Sequential()
    shared_gru = GRU(64) #, input_shape=(X_train_a.shape[1], X_train_b.shape[2]))
        
    # When we reuse the same layer instance multiple times, the weights of the layer
    # are also being reused (it is effectively *the same* layer)
    encoded_a = shared_gru(team_a)
    encoded_b = shared_gru(team_b)

    # We can then concatenate the two vectors:
    gru_out = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)
    home_input = Input(shape=(1,), name='home_input')
    merged_vector = keras.layers.concatenate([gru_out, home_input])

    # And add a logistic regression on top
    predictions = Dense(1, activation='sigmoid')(merged_vector)

    model = Model(inputs=[team_a, team_b, home_input], outputs=predictions)

    model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    model.fit([X_train_a, X_train_b, home_input_train], y_train, epochs=3, batch_size=32)   
    
#         print(model.summary())
#     model.fit(X_train, y_train, epochs=3, batch_size=32)
    
    # Final evaluation of the model
    scores_train = model.evaluate([X_train_a, X_train_b, home_input_train], y_train, verbose=0)
    scores = model.evaluate([X_test_a, X_test_b, home_input_test], y_test, verbose=0)
#     preds_train = model.predict_classes(X_test)

    
    print("Train Accuracy: %.2f%%" % (scores_train[1]*100))
    print("Test Accuracy: %.2f%%" % (scores[1]*100))

    #     preds = model. ([X_test_a, X_test_b, home_input_test])
    
#     print(np.unique(preds_train))
#     print(np.unique(preds))

train_sequential(Xa_2000, Xb_2000, home_inputs_2000, y_2000,
                 Xa_2001, Xb_2001, home_inputs_2001, y_2001)



Epoch 1/3
2928/2928 [==============================] - 4s 1ms/step - loss: 0.6762 - acc: 0.6072A: 3s - loss: 0
Epoch 2/3
2928/2928 [==============================] - 2s 789us/step - loss: 0.6737 - acc: 0.6062
Epoch 3/3
2928/2928 [==============================] - 2s 796us/step - loss: 0.6717 - acc: 0.6083
Train Accuracy: 60.66%
Test Accuracy: 57.55%


In [ ]:
def train_sequential_basic(X_train, y_train, X_test=None, y_test=None, train_ratio = 0.9):
    numpy.random.seed(12)
    
    if X_test is None:    
        train_ratio = 0.9
        train_max_idx = int(train_ratio*X.shape[0])

        X_test = X_train[train_max_idx:]
        y_test = y[train_max_idx:]
        
        X_train = X_train[:train_max_idx]
        y_train = y[:train_max_idx]
        
    model = Sequential()
    model.add(GRU(100, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Input(input_shape=(X_train.shape[0], 1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
    print(model.summary())
    model.fit(X_train, y_train, epochs=3, batch_size=32)
    
    # Final evaluation of the model
    scores_train = model.evaluate(X_train, y_train, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    preds_train = model.predict_classes(X_test)
    preds = model.predict_classes(X_test)
    
    print(np.unique(preds_train))
    print(np.unique(preds))
    
    print("Train Accuracy: %.2f%%" % (scores_train[1]*100))
    print("Test Accuracy: %.2f%%" % (scores[1]*100))

train_sequential(X_2000, y_2000, X_2001, y_2001)



In [183]:
np.unique(model.predict_classes(X_test))

array([1])

In [248]:
np.bincount(y_test.astype(int))#/y_test.shape[0]

array([ 728, 1014], dtype=int64)

In [226]:
def get_sin_cos_sequences(time_steps=20, count_rows=500, f1=1, f2=1.5, a1=1, a2=0.8):
    """
    Generate some dummy data for testing sequence to sequence clustering
    - Sine and cosine curves, with parameters provided as function inputs
    """
    
    x = np.arange(time_steps)
    
    _, X = np.mgrid[0:count_rows//2, 0:time_steps:1]
    targets = np.zeros(count_rows)
    targets[:count_rows//2] = 1
    rands1 = f1 * 2 * np.pi * np.random.rand(count_rows//2, 1)
    rands2 = f2 * 2 * np.pi * np.random.rand(count_rows//2, 1)
    
    feature_length = 2
    
    # sequences1a shape: 250, 50
    sequences1a =   a1 * np.sin(f1*X*rands1/time_steps)
    sequences1b =   np.random.rand(1) * a1 * np.sin(f1*X*rands1/time_steps)
    
    # Want sequences to be 250, 50, 2
    sequences1 = np.concatenate(([np.ravel(sequences1a)], [np.ravel(sequences1b)]), axis=0) 
    print(sequences1.shape)
    sequences1 = sequences1.T.reshape(count_rows//2, time_steps, feature_length)
    print(sequences1.shape)

    sequences2a =   a2 * np.cos(f2*X*rands2/time_steps)
    sequences2b =   np.random.rand(1) * a2* np.cos(f2*X*rands2/time_steps)
    sequences2 = np.concatenate(([np.ravel(sequences2a)], [np.ravel(sequences2b)]), axis=0) 
    sequences2 = sequences2.T.reshape(count_rows//2, time_steps, feature_length)
    
    sequences = np.concatenate((sequences1, sequences2), axis=0)
    print(sequences.shape)
    
#     for i in range(count_rows//2):
#         plt.plot(x, sequences1[i], alpha=0.05)
#         plt.plot(x, sequences1b[i], alpha=0.05)
#         plt.plot(x, sequences2[i], alpha=0.05)

#     plt.xlabel('sample(n)')
#     plt.ylabel('voltage(V)')
#     plt.show()
    
#     sequences = np.vstack([sequences1, sequences2])
    return sequences, targets #sequences.reshape(count_rows, time_steps, 1)
#     
X_dummy, y_dummy = get_sin_cos_sequences()

(2, 5000)
(250, 20, 2)
(500, 20, 2)


In [244]:
numpy.random.seed(12)

train_ratio = 0.9
train_max_idx = int(train_ratio*X.shape[0])

# X_train = X[:train_max_idx]
# y_train = y[:train_max_idx]
# X_test = X[train_max_idx:]
# y_test = y[train_max_idx:]
print(X.shape)

def train_sequential_dummy(X, y):
    model = Sequential()
    model.add(GRU(100, input_shape=(X.shape[1],X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
    print(model.summary())
    model.fit(X, y, epochs=3, batch_size=32)
    # Final evaluation of the model
    scores = model.evaluate(X, y, verbose=0)
    preds = model.predict(X)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
train_sequential_dummy(X_dummy, y_dummy)

(1754, 20, 5)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_26 (GRU)                 (None, 100)               30900     
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 101       
Total params: 31,001
Trainable params: 31,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
500/500 [==============================] - 4s 8ms/step - loss: 0.6399 - acc: 0.6300
Epoch 2/3
500/500 [==============================] - 0s 566us/step - loss: 0.2560 - acc: 0.9240
Epoch 3/3
500/500 [==============================] - 0s 586us/step - loss: 0.0571 - acc: 0.9840
Accuracy: 100.00%
